In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import time
import pymongo

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
db = client.mars_db
collection = db.items

In [4]:
url = 'https://mars.nasa.gov/news/'
response = requests.get(url)

In [5]:
headlines_soup = bs(response.text, 'lxml')

In [6]:
headline = headlines_soup.find('div', class_ = 'content_title').text.strip()

In [7]:
print(headline)

Opportunity Hunkers Down During Dust Storm


In [8]:
news = headlines_soup.find('div', class_ = 'rollover_description_inner').text.strip()

In [9]:
print(news)

It's the beginning of the end for the planet-encircling dust storm on Mars. But it could still be weeks, or even months, before skies are clear enough for NASA's Opportunity rover to recharge its batteries and phone home.


In [10]:
from selenium import webdriver
from splinter import Browser
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [11]:
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [12]:
browser.click_link_by_id('full_image')

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=70.0.3538.77)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.14.0 x86_64)


In [ ]:
time.sleep(2)

In [ ]:
html = browser.html
image_soup = bs(html, "lxml")
img_url = image_soup.find('img', class_ = 'fancybox-image')['src']

In [ ]:
featured_img_url = "https://www.jpl.nasa.gov" + img_url

In [ ]:
featured_img_url

In [ ]:
url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(url)

In [ ]:
weather_soup = bs(response.text, 'html.parser')

In [ ]:
mars_weather_tweet = weather_soup.find_all('div', class_ = "js-tweet-text-container")

In [ ]:
for tweet in mars_weather_tweet:
    if tweet.text.strip().startswith('Sol'):
        mars_weather = tweet.text.strip()

In [ ]:
print(mars_weather)

# facts

In [ ]:
url = 'https://space-facts.com/mars/'
browser.visit(url)

In [ ]:
mars_df = pd.read_html(url)
mars_df = (mars_df[0])

In [ ]:
mars_df.columns = ["Description", "Value"]


In [ ]:
mars_df

In [ ]:
mars_df = mars_df.to_html(classes='mars')
table_data = mars_df.replace('\n', ' ')
table_data

# database

In [ ]:
post = {
    'headline': headline, 
    'news': news, 
    'featured': featured_img_url,
    'weather': mars_weather,
    'facts': table_data
}
collection.insert_one(post)

In [42]:
webdatas = db.items.find()

for webdata in webdatas:
    print(webdata)

{'_id': ObjectId('5bdcfea95aa45abbfd07f726'), 'headline': 'Opportunity Hunkers Down During Dust Storm', 'news': "It's the beginning of the end for the planet-encircling dust storm on Mars. But it could still be weeks, or even months, before skies are clear enough for NASA's Opportunity rover to recharge its batteries and phone home.", 'featured': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA20318_ip.jpg', 'weather': 'Sol 2172 (2018-09-15), high -14C/6F, low -73C/-99F, pressure at 8.93 hPa, daylight 05:43-18:00', 'facts': '<table border="1" class="dataframe mars">   <thead>     <tr style="text-align: right;">       <th></th>       <th>Description</th>       <th>Value</th>     </tr>   </thead>   <tbody>     <tr>       <th>0</th>       <td>Equatorial Diameter:</td>       <td>6,792 km</td>     </tr>     <tr>       <th>1</th>       <td>Polar Diameter:</td>       <td>6,752 km</td>     </tr>     <tr>       <th>2</th>       <td>Mass:</td>       <td>6.42 x 10^23 kg (10.7% Earth)</